# Dask and pandas

In [1]:
!pip install dask
!pip install cloudpickle
!pip install "dask[dataframe]"
!pip install "dask[complete]"

In [23]:
!pip show dask

Name: dask
Version: 2022.2.1
Summary: Parallel PyData with Task Scheduling
Home-page: https://github.com/dask/dask/
Author: 
Author-email: 
License: BSD
Location: c:\users\hp\anaconda3\lib\site-packages
Requires: toolz, partd, packaging, cloudpickle, pyyaml, fsspec
Required-by: intake, distributed, datashader


In [24]:
import os
import pandas as pd
import dask
from dask.distributed import Client
import dask.dataframe as dd
import numpy as np
import dask.multiprocessing

In [25]:
dd=dd.read_csv("tested.csv")
dd

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
npartitions=1,,,,,,,,,,,,
,int64,int64,int64,object,object,float64,int64,int64,object,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...


In [26]:
dd.shape

(Delayed('int-403d80f0-96c1-4cf1-9358-4467ce1b5725'), 12)

In [31]:
dd.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [32]:
dd["Name"].head(1)

0    Kelly, Mr. James
Name: Name, dtype: object

In [35]:
dd.tail(3)

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+--------+----------+
| Column | Found  | Expected |
+--------+--------+----------+
| Cabin  | object | float64  |
+--------+--------+----------+

The following columns also raised exceptions on conversion:

- Cabin
  ValueError("could not convert string to float: 'B45'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'Cabin': 'object'}

to the call to `read_csv`/`read_table`.

In [36]:
dd.head()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+--------+----------+
| Column | Found  | Expected |
+--------+--------+----------+
| Cabin  | object | float64  |
+--------+--------+----------+

The following columns also raised exceptions on conversion:

- Cabin
  ValueError("could not convert string to float: 'B45'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'Cabin': 'object'}

to the call to `read_csv`/`read_table`.

In [49]:
dd.max()

Dask Series Structure:
npartitions=1
Age       object
Ticket       ...
dtype: object
Dask Name: dataframe-max-agg, 3 tasks

In [50]:
dd.min()

Dask Series Structure:
npartitions=1
Age       object
Ticket       ...
dtype: object
Dask Name: dataframe-min-agg, 3 tasks

In [45]:
dd["Survived"].max()

dd.Scalar<series-..., dtype=int32>

In [46]:
dd["Survived"].min()

dd.Scalar<series-..., dtype=int32>

In [16]:
df=pd.read_csv("tested.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [37]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [38]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,0,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [39]:
df.shape

(418, 12)

In [40]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [41]:
df.max()

C:\Users\hp\AppData\Local\Temp\ipykernel_2304\1151452817.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.max()


PassengerId                                 1309
Survived                                       1
Pclass                                         3
Name           van Billiard, Master. Walter John
Sex                                         male
Age                                         76.0
SibSp                                          8
Parch                                          9
Ticket                               W.E.P. 5734
Fare                                    512.3292
Embarked                                       S
dtype: object

In [42]:
df.min()

C:\Users\hp\AppData\Local\Temp\ipykernel_2304\3962516015.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.min()


PassengerId                              892
Survived                                   0
Pclass                                     1
Name           Abbott, Master. Eugene Joseph
Sex                                   female
Age                                     0.17
SibSp                                      0
Parch                                      0
Ticket                                110469
Fare                                     0.0
Embarked                                   C
dtype: object

In [47]:
df["Survived"].max()

1

In [48]:
df["Survived"].min()

0